In [4]:
from collections import Counter
#L1=[10,20,36,36,2,3,3,69,10,20,25,20,36,10]
#count=Counter(L1)
#print(count.most_common(1))
#print(count.most_common(2))
#count.most_common(2)[1][1]

In [5]:
from langdetect import detect
import googletrans
from googletrans import Translator
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.util import ngrams
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import contractions
import yake
from rake_nltk import Rake
from unidecode import unidecode
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import SilhouetteVisualizer

In [6]:
data=pd.read_csv('dataset_es_train.csv')
data.head(2)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0491108,product_es_0296024,reviewer_es_0999081,1,Nada bueno se me fue ka pantalla en menos de 8...,television Nevir,es,electronics
1,es_0869872,product_es_0922286,reviewer_es_0216771,1,"Horrible, nos tuvimos que comprar otro porque ...",Dinero tirado a la basura con esta compra,es,electronics


In [7]:
data.shape

(200000, 8)

In [8]:
sample = data.sample(100)
sample.reset_index(drop=True,inplace=True)
sample

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0565694,product_es_0752352,reviewer_es_0206726,2,Desde el primer día las puse y han funcionado ...,Han durado 1 año,es,home
1,es_0777440,product_es_0578815,reviewer_es_0125595,3,Por el precio que tiene cumple las expectativas,Precio - calidad - bien,es,wireless
2,es_0004790,product_es_0445877,reviewer_es_0340121,4,"Calidad y precio estupendo, venía todo muy bie...",Contenta,es,toy
3,es_0040618,product_es_0206868,reviewer_es_0659304,3,"Son soportes algo endebles, pero para muñecas ...",Suficientes,es,toy
4,es_0252341,product_es_0323947,reviewer_es_0478834,4,Para windows hay que instalar el driver del CD...,Chip correcto,es,pc
...,...,...,...,...,...,...,...,...
95,es_0646398,product_es_0426925,reviewer_es_0339032,4,El pedido se recibió pronto y correctamente. T...,Cumple lo esperado,es,home_improvement
96,es_0361956,product_es_0537299,reviewer_es_0958798,1,No he recibido el paquete todavía! No sé si se...,Imposible valoración,es,toy
97,es_0625797,product_es_0737551,reviewer_es_0184688,2,Me esperaba o tipo de calcetines .son finos no...,Son muy finos,es,apparel
98,es_0477922,product_es_0915368,reviewer_es_0310537,5,"Se trata de un pulsioximetro muy útil, funcion...",Práctico y cómodo,es,personal_care_appliances


In [9]:
df=sample[['review_body','stars']]
df

,review_body,stars
0,Desde el primer día las puse y han funcionado ...,2
1,Por el precio que tiene cumple las expectativas,3
2,"Calidad y precio estupendo, venía todo muy bie...",4
3,"Son soportes algo endebles, pero para muñecas ...",3
4,Para windows hay que instalar el driver del CD...,4
...,...,...
95,El pedido se recibió pronto y correctamente. T...,4
96,No he recibido el paquete todavía! No sé si se...,1
97,Me esperaba o tipo de calcetines .son finos no...,2
98,"Se trata de un pulsioximetro muy útil, funcion...",5


In [10]:
# language detection
def lang_detect(data):
    lang = detect(data)
    return lang

In [11]:
print(df['review_body'][1])
lang_detect(df['review_body'][1])

Por el precio que tiene cumple las expectativas


'es'

In [12]:
df['language']=df.review_body.apply(lang_detect)
df

KeyboardInterrupt: 

In [ ]:
df.language.unique()

In [ ]:
df.language.value_counts()

In [ ]:
# language translation

def lang_trans(data):
    translor = Translator()
    translated_text = translor.translate(data)
    return translated_text.text

In [ ]:
print(df.review_body[2])
lang_trans(df.review_body[2])

In [ ]:
df['translated_review']= df.review_body.apply(lang_trans)
df

## EDA

In [ ]:
def expand_text(data):
    expanded_text=contractions.fix(data)
    return expanded_text

stopword_list = stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('nor')
stopword_list.remove('not')

def clean_data(data):
    tokens=word_tokenize(data)
    clean_text=[word.lower() for word in tokens if (word not in punctuation) and (word.lower() not in stopword_list) and (len(word)>2) and (word.isalpha())]
    return ' '.join(clean_text)

In [ ]:
clean_text = df.translated_review.apply(expand_text)
clean_text = clean_text.apply(clean_data)

In [ ]:
clean_text

In [ ]:
print(clean_text[6])

In [ ]:
print(df.translated_review[6])

## 1.Ngram Creation

In [ ]:
def ngram_extractor(data,ngram_range):
    tokens=word_tokenize(data)
    ngram= ngrams(tokens,ngram_range)
    ngram_list1=[]
    
    for ngram1 in ngram:
        ngram_list1.append(' '.join(ngram1))
    return ngram_list1     

In [ ]:
list_unigrams = clean_text.apply(lambda x:ngram_extractor(x,1))
final_unigram=[]

for unigram in list_unigrams:
    final_unigram.extend(unigram)
cnt=Counter(final_unigram).most_common(25)

In [ ]:
cnt=Counter(final_unigram).most_common(25)
cnt

In [ ]:
list_bigrams = clean_text.apply(lambda x:ngram_extractor(x,2))
final_bigram=[]

for bigram in list_bigrams:
    final_bigram.extend(bigram)
cnt2=Counter(final_bigram).most_common(25)

In [ ]:
cnt2=Counter(final_bigram).most_common(25)
cnt2

In [ ]:
list_trigrams = clean_text.apply(lambda x:ngram_extractor(x,3))
final_trigram=[]

for trigram in list_trigrams:
    final_trigram.extend(trigram)
cnt3=Counter(final_trigram).most_common(25)
cnt3

## 2.Wordcloud generation

In [ ]:
def wordcloud(data,column):
    df_= data[column].str.cat(sep=' ')
    text =' '.join([word for word in df_.split()])
    wordcloud = WordCloud(width=700 ,height=500 ,background_color='white').generate(text)
    plt.figure(figsize = (20,50))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()
  
wordcloud(df,'translated_review')

## 3.Keyphrase extraction

### YAKE

Syntax


i/p :-

from yake import KeywordExtractor

text = "This is a sample text for testing YAKE keyword extraction function in Python"
kw_extractor = KeywordExtractor()
keywords = kw_extractor.extract_keywords(text)

print(keywords)

o/p

[('YAKE keyword extraction function', 0.1560054334917623), ('Python', 0.1686488256612227), ('sample text', 0.2113766985092226), ('testing', 0.2842842231669544), ('function', 0.30201086698352487), ('keyword extraction', 0.34850109522579016)]



In [ ]:
def yake_extracter(data):
    keyword_extractor = yake.KeywordExtractor()
    keywords = keyword_extractor.extract_keywords (data)
    keyword_list = []
    
    for kw in keywords:
        keyword_list.append(kw[0])
    return keyword_list

keywords=df.translated_review.apply(yake_extracter)
all_keywords = []

for kw in keywords:
    all_keywords.extend(kw)
    
cnt_kw=Counter(all_keywords).most_common(1500)
cnt_kw
    

### RAKE

Syntax


i/p :-

from rake_nltk import Rake

text = "This is a sample text for testing RAKE keyword extraction function in Python"
r = Rake()
r.extract_keywords_from_text(text)
keywords = r.get_ranked_phrases()

print(keywords)

o/p :- 

['testing rake keyword extraction function', 'sample text', 'python', 'keyword extraction function', 'rake keyword extraction function']

In [ ]:
def rake_extracter(data):
    keyword_extractor = Rake()
    keyword_extractor.extract_keywords_from_text(data)
    return keyword_extractor.get_ranked_phrases()

rake_keywords=df.translated_review.apply(rake_extracter)
all_keywords = []

for kw_rake in rake_keywords:
    all_keywords.extend(kw_rake)
    
cnt_kw_rake=Counter(all_keywords).most_common(1500)
cnt_kw_rake

## Preprocessing

In [ ]:
# Remove spaces, new lines
def remove_spaces(data):
    clean_text= data.replace('\\n','').replace('\t','').replace('\\','')
    return clean_text

#contraction mapping
def expand_text(data):
    expanded_text=contractions.fix(data)
    return expanded_text

# handling accented characters
def handling_accented(data):
    fixed_text=unidecode(data)
    return fixed_text

#cleaning
stopword_list = stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('nor')
stopword_list.remove('not')

def clean_data(data):
    tokens=word_tokenize(data)
    clean_text=[word.lower() for word in tokens if (word not in punctuation) and (word.lower() not in stopword_list) and (len(word)>2) and (word.isalpha())]
    return ' '.join(clean_text)

#autocorrect
#def autocorrection(data):
    #spell=Speller(lang='en')
    #corrected_text=spell(data)
    #return corrected_text

# lemmatization

def lemmatization(data):
    lemmatizer = WordNetLemmatizer()
    final_data = []
    
    for word in data:
        lemmatized_word = lemmatizer.lemmatize(word)
        final_data.append(lemmatized_word)
        return " ".join(final_data)
    

In [ ]:
test = df[(df.stars<3)].reset_index(drop=True)

In [ ]:
test

In [ ]:
clean_text_train= df.translated_review.apply(remove_spaces)

clean_text_train= clean_text_train.apply(expand_text)

clean_text_train= clean_text_train.apply(handling_accented)

clean_text_train= clean_text_train.apply(clean_data)

#clean_text_train= clean_text_train.apply(lemmatization)

In [ ]:
clean_text_train

In [ ]:
#Text vectorizer

#Count vectorizer

count_vec= CountVectorizer()
bow= count_vec.fit_transform(clean_text_train).A
pd.DataFrame(bow,columns= count_vec.get_feature_names())

In [ ]:
#tfidf vectorizer

tfidf_vec = TfidfVectorizer()
tfidf = tfidf_vec.fit_transform(clean_text_train).A
pd.DataFrame(tfidf,columns= tfidf_vec.get_feature_names())

In [ ]:
print(clean_text_train[1])

In [ ]:
#word2vec

sent=clean_text_train.tolist()
splitted_text = [x.split() for x in sent]
print(splitted_text)

In [ ]:
word_2vec_model= Word2Vec(splitted_text,min_count=2, window=3)

In [ ]:
#saving the model
word_2vec_model.save('word2vec.model')

In [ ]:
def vectorizer(list_of_docs,model):
    feature=[]
    
    for rew in list_of_docs:
        zero_vector= np.zeros(model.vector_size)
        vector=[]
        for word in rew:
            if word in model.wv:
                try:
                    vector.append(model.wv[word]) # model.wv['bad']
                except KeyError:
                    continue
                    
        if vector:
            vector=np.asarray(vector)
            avg_vec=vector.mean(axis=0)
            feature.append(avg_vec)
            
        else:
            feature.append(zero_vector)
    return feature
                    

In [ ]:
vectorized_docs=vectorizer(splitted_text,word_2vec_model)

In [ ]:
vectorized_docs

In [ ]:
len(vectorized_docs)

In [ ]:
x_emb=np.array(vectorized_docs)

In [ ]:
x_emb

In [ ]:
len(x_emb)

In [ ]:
def build_kmeans (clusters,data):
    kmeans_model=KMeans(n_clusters=clusters)
    y_pred=kmeans_model.fit_predict(data)
    return kmeans_model, y_pred

In [ ]:
#kmeans_count_vectorizer
kmeans_model_count, count_pred = build_kmeans (3,bow)

In [ ]:
count_pred

In [ ]:
len(count_pred)

In [ ]:
#kmeans_tfidf_vectorizer
kmeans_model_tfidf, tfidf_pred = build_kmeans (3,tfidf)

In [ ]:
tfidf_pred

In [ ]:
#kmeans_word2vec_vectorizer
kmeans_model_word2vec, word2vec_pred = build_kmeans (3,x_emb)

In [ ]:
word2vec_pred 

In [ ]:
print(f'silhouette score with kmeans-count : {silhouette_score(bow, count_pred)}')

print(f'silhouette score with kmeans-tfidf : {silhouette_score(tfidf, tfidf_pred)}')

print(f'silhouette score with kmeans-word2vec : {silhouette_score(x_emb, word2vec_pred)}')


In [ ]:
def visualize_silhouette (data,model,title):
    visualizer= SilhouetteVisualizer(model, colors='yellowbrick')
    visualizer.fit(data)
    plt.title(f'Silhouette visualizer for {title}')

In [ ]:
visualize_silhouette (bow, kmeans_model_count,'kmeans_count Vectorizer')

In [ ]:
visualize_silhouette (tfidf, kmeans_model_tfidf,'kmeans_tfidf Vectorizer')

In [ ]:
visualize_silhouette (x_emb, kmeans_model_word2vec,'kmeans_word2vec Vectorizer')

## Model Building

In [ ]:
df

In [ ]:
df['Target']=word2vec_pred

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df.translated_review,df.Target,test_size=0.25,random_state=1)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
count_mnb = MultinomialNB()
count_mnb.fit(x_train,y_train)
predict_count=count_mnb.predict(clean_text_test)
accuracy_count=accuracy_score(y_test,predict_count)*100
accuracy_count